# Creación y ejecución de un modelo de MODFLOW 2005 con flopy
## Luis M. de la Cruz
<font color=#FF0000>**Última actualización: mar may 26 17:27:14 CDT 2020**</font><br>

En este ejemplo se muestra como usar las herramientas de flopy para crear un modelo de MODFLOW 2005 y ejecutarlo. Se verá cómo crear los datos de entrada y cómo definir los parámetros de salida de la simulación.


## Paso 1. Importar bibliotecas

Importamos las bibliotecas `numpy` y `flopy`.

In [1]:
import numpy as np
import flopy

flopy is installed in /home/luiggi/anaconda3/lib/python3.7/site-packages/flopy


## Paso 2. Inicializaciones
Definimos la ruta donde se encuentra el ejecutable de MODFLOW 2005 y el nombre de la simulación.

In [2]:
# WINDOWS (descomente la siguiente línea)
#mf2005_exe = 'C:\\Users\\luiggi\\Desktop\\MODFLOW\\MF2005.1_12\\bin\\mf2005.exe'

# LINUX (desocomente la siguiente línea)
mf2005_exe = '/home/luiggi/GitSites/MODFLOW/MF2005.1_12u/bin/mf2005'
# Definimos el nombre de la simulación
modelname = 'ejem_01_MF2005'

print(' Ruta hacia el ejecutable de MODFLOW 2005 : {} '.format(mf2005_exe))
print(' Nombre de esta simulación: {}'.format(modelname))

 Ruta hacia el ejecutable de MODFLOW 2005 : /home/luiggi/GitSites/MODFLOW/MF2005.1_12u/bin/mf2005 
 Nombre de esta simulación: ejem_01_MF2005


## Paso 3. Creación del modelo de modflow 2005

In [3]:
mf = flopy.modflow.Modflow(modelname, exe_name = mf2005_exe) 

In [4]:
type(mf)

flopy.modflow.mf.Modflow

In [5]:
print(mf)

MODFLOW 0 layer(s) 0 row(s) 0 column(s) 0 stress period(s)


## Paso 4. Modelo discreto.

Definición del dominio y de la malla del modelo, para crear un objeto para la discretización (<a href="https://modflowpy.github.io/flopydoc/mfdis.html">ModflowDis</a>):

In [6]:
Lx = 1000.
Ly = 1000.
ztop = 0.
zbot = -50.
nlay = 1
nrow = 20
ncol = 15
delr = Lx/ncol
delc = Ly/nrow
delv = (ztop - zbot) / nlay
botm = np.linspace(ztop, zbot, nlay + 1) 

In [7]:
print(delc, delr, delv, botm, sep='\n')

50.0
66.66666666666667
50.0
[  0. -50.]


In [8]:
# Modelo discreto
dis = flopy.modflow.ModflowDis(mf, 
                               nlay, nrow, ncol, 
                               delr = delr, delc = delc, 
                               top = ztop, botm = botm[1:])

In [ ]:
print(mf)

In [ ]:
type(dis) 

In [ ]:
print(dis)

## Paso 3. Basic Package
Creación de un objeto de flopy para representar el *MODFLOW Basic Package* (Más detalles en <a href="https://modflowpy.github.io/flopydoc/mfbas.html">BAS</a>). 

Para el modelo de este ejemplo se asignan valores constantes a la carga (head values) de 10.0 y 0.0 a la primera y última columna del modelo, en todas las capas. 


In [9]:
ibound = np.ones((nlay, nrow, ncol))#, dtype=np.int32)
ibound[:, :, 0] = -1 
ibound[:, :, -1] = -1

ibound[:,4:8,4:8] = -1 # Definiendo un obstáculo

strt = np.ones((nlay, nrow, ncol))#, dtype=np.float32)
strt[:, :, 0] = 10.
strt[:, :, -1] = 0.
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)

In [ ]:
print(ibound)

In [ ]:
print(strt)

In [ ]:
type(bas)

In [ ]:
print(bas)

## Paso 4. Layer-Property Flow Package

Asignamos propiedades a las capas del modelo usando el *Layer-Property Flow Package* (<a href="https://modflowpy.github.io/flopydoc/mflpf.html">mflpf</a>). Se asignan valores de 10.0 a la conductividad hidráhulica, horizontal y verticalmente. 

**NOTA**: Como no se especifica un valor para `laytyp`, Flopy usa el valor por omisión de 0, lo cual significa que este modelo será confinado.

In [10]:
lpf = flopy.modflow.ModflowLpf(mf, hk=10., vka=10., ipakcb=53)

In [ ]:
type(lpf)

In [ ]:
print(lpf)

## Paso 5. Parámetros para la salida

Se definen los parámetros para la salida de la simulación (<a href="https://modflowpy.github.io/flopydoc/mfoc.html">mfoc</a>)

**NOTA**: se utiliza un diccionario para definir que salida será guardada para los correspondientes periodos de *stress* y pasos de tiempo. En este caso, la tupla `(0,0)` significa que el periodo 1 y el tiempo 1 serán guardados. 

In [11]:
spd = {(0, 0): ['print head', 'print budget', 'save head', 'save budget']}
oc = flopy.modflow.ModflowOc(mf, stress_period_data=spd, compact=True)

In [ ]:
print(type(spd), type(oc), sep='\n')

In [ ]:
print(spd)

In [ ]:
print(oc)

## Paso 6. Algoritmo de solución

Definición del algoritmo de solución. En este caso se usa *Preconditioned Conjugate Gradient Package* 
(<a href="https://modflowpy.github.io/flopydoc/mfpcg.html">mfpcg</a>). En este caso se usarán todos los valores por omisión.

In [12]:
pcg = flopy.modflow.ModflowPcg(mf)

In [ ]:
type(pcg)

In [ ]:
print(pcg)

## Paso 7. Escritura de datos de entrada

Escribir los datos de entrada del modelo en un archivo se logra de la manera siguiente:

In [13]:
mf.write_input()

## Paso 8. Ejecución del modelo

El objeto del modelo, en este caso `mf`, tiene un método para ejecutar el modelo. Para que esto funcione, el ejecutable de MODFLOW debe estar localizado dentro de una ruta accesible. En este caso, la ruta completa al ejecutable está identificada con `mf2005_exe`.

In [14]:
success, buff = mf.run_model()

FloPy is using the following  executable to run the model: /home/luiggi/GitSites/MODFLOW/MF2005.1_12u/bin/mf2005

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: ejem_01_MF2005.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2020/08/25 16:14:03

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2020/08/25 16:14:03
 Elapsed run time:  0.018 Seconds

  Normal termination of simulation


### Siguiente paso:

Continuar con el ejercicio de visualización en: <a href="./MF2005-Vis1.ipynb">MF2005-Vis1 </a>

# Referencias útiles:

<a href=https://modflowpy.github.io/flopydoc/tutorials.html>Flopy Tutorials</a>

<a href=https://github.com/modflowpy/flopy>Github: Flopy</a>

- <a href=https://github.com/modflowpy/flopy/tree/develop/examples/Notebooks>flopy/examples/Notebooks/</a>

<a href=https://www2.hawaii.edu/~jonghyun/classes/S18/CEE696/>CEE 696: Optimization in Groundwater Engineering</a>
